In [171]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.optimizers import Optimizer
from tensorflow.keras.models import Sequential


In [172]:
train=pd.read_csv("veriler\datathon\\train.csv")
sample=pd.read_csv("veriler\datathon\\sample_solution.csv")
test=pd.read_csv("veriler\datathon\\testFeatures.csv")

In [173]:
sutunisimler=["tarih","urun","besin","kategori","fiyat","uretimyeri","market","sehir"]
train.columns=sutunisimler

In [174]:
one_hot = pd.get_dummies(train[['urun','kategori','uretimyeri','market','sehir']])
train = train.drop(['urun','kategori','uretimyeri','market','sehir','besin'],axis = 1)
train = train.join(one_hot)

In [175]:
train["tarih"]=train["tarih"].str.replace("-","").astype(float)

In [176]:
y=train.iloc[:,1:2]
x = train.drop(['fiyat'],axis = 1)

In [177]:
y.head()

,fiyat
0,33.41
1,11.22
2,11.12
3,7.02
4,13.14


In [178]:
x.head()

,tarih,urun_Armut,urun_Bakla-kuru,urun_Bakla-taze,urun_Bamya,urun_Barbunya-kuru,urun_Bezelye-taze,urun_Biber-yeşil,urun_Brüksel laha.,urun_Bulgur,...,market_C,market_M,sehir_A,sehir_B,sehir_C,sehir_D,sehir_E,sehir_F,sehir_G,sehir_H
0,20160101.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
1,20160101.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
2,20160101.0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
3,20160101.0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
4,20160101.0,0,0,1,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0


In [179]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=0)

In [180]:
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [181]:
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))
step_size = 99

In [182]:
model=Sequential()
model.add(LSTM(256,input_shape=(1, step_size), return_sequences = True))
model.add(LSTM(128))
model.add(Dense(1))

In [183]:
model.compile(optimizer="Adam",loss="mse")

In [184]:
model.fit(x_train,y_train,epochs=50,validation_split=0.1)

Epoch 1/50
4288/4288 [==============================] - 44s 10ms/step - loss: 34.2890 - val_loss: 7.0327
Epoch 2/50
4288/4288 [==============================] - 43s 10ms/step - loss: 5.1647 - val_loss: 4.6799
Epoch 3/50
1835/4288 [===========>..................] - ETA: 48s - loss: 4.3012

KeyboardInterrupt: 

In [ ]:
history=pd.DataFrame(model.history.history)
history.plot()

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
testTahminleri=model.predict(x_test)
len(testTahminleri)

In [ ]:
tahmin=pd.DataFrame(y_test.values)
testTahminleri= pd.Series(testTahminleri.reshape(75082,))
tahminDf=pd.concat([tahmin,testTahminleri],axis=1)
tahminDf.columns=["Gerçek Değer","Test Sonucu"]

In [ ]:
tahminDf.head(20)

In [ ]:
model.save('veriler\datathon\\10024')

In [ ]:
new_model = tf.keras.models.load_model('veriler\datathon\\10024')

In [ ]:
test=test.drop('id',axis=1)

In [ ]:
sutunisimler=["tarih","urun","besin","kategori","uretimyeri","market","sehir"]
test.columns=sutunisimler

In [ ]:
one_hot = pd.get_dummies(test[['urun','kategori','uretimyeri','market','sehir']])
test = test.drop(['urun','kategori','uretimyeri','market','sehir','besin'],axis = 1)
test= test.join(one_hot)

In [ ]:
test["tarih"]=test["tarih"].str.replace("-","").astype(float)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(test)
test = scaler.transform(test)

In [ ]:
test = np.reshape(test, (test.shape[0], 1, test.shape[1]))

In [ ]:
testTahminleri=model.predict(test)
len(testTahminleri)

In [ ]:
testTahminleri= pd.Series(testTahminleri.reshape(45504,))

In [ ]:
testTahminleri.to_csv('veriler\datathon\\sonuc2.csv')

In [ ]:
deneme=pd.read_csv('veriler\datathon\\sonuc2.csv')

In [ ]:
sutunisimler=["id","ürün fiyatı"]
deneme.columns=sutunisimler

In [ ]:
deneme.to_csv('veriler\datathon\sonuclar\\sonucLSTM.csv',index=False)